<a href="https://colab.research.google.com/github/ytyimin/scm518/blob/main/Tomato_Transportation_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tomatoes Transportation Routes

## Objective and Prerequisites

This tomato transhipment problem shows you how to ship tomato products from multiple plants to multiple destinations via transfer stations, where shipments can be bi-directional among plants and destinations. The objectives of the transsioment problem are:

* Meet demand while ensuring plant capacities are met,
* Minimize the overall transshipment cost of satisfying all demands, and
* Ensure that the shipments are valid, i.e., cannot transport along certain routes among plants and destinations.

This modeling example is at the introductory level, where we assume that you know Python and that you have some knowledge of how to build mathematical optimization models.

---
## Problem Description

![picture](https://drive.google.com/uc?id=1xnG76mkx-M5MKYfqyCjjw7FfLzXz_oKC)

The RedBrand Company produces a tomato product at three plants. This product can be shipped directly to the company’s two customers, or it can first be shipped to the company’s two warehouses and then to the customers. 

Nodes 1, 2, and 3 represent the plants (these are the origins, denoted by S for supplier), nodes 4 and 5 represent the warehouses (these are the transshipment points, denoted by T), and nodes 6 and 7 represent the customers (these are the destinations, denoted by D). Note that some shipments are allowed among plants, among warehouses, and among customers. Also, some arcs have arrows on both ends. This means that flow is allowed in either direction.

The supply capacity for plants 1, 2, and 3 is 200, 300, and 100 tons, respectively. The dmand for customers 6 and 7 are 400 and 180 tons, respectively. 

The following table lists the shipping cost among different nodes. Note that not all nodes are connected, so a sparse representation is more desirable for model setup. 
	
| From / To |	1	    | 2	    | 3 	  | 4	   | 5	  | 6	   | 7	  |
| ---     | ---   | ---   | ---   | ---  | ---  | ---  | ---  | 
|1	      | -	    | 5.0	| 3.0	| 5.0 | 5.0 | 20.0 | 20.0  | 
|2	      | 9.0	| -	| 9.0	    | 1.0 |	1.0	  | 8.0 | 15.0 |	
|3	      | 0.4     |	8.0	|-	    |1.0	   |0.5	    | 10.0 | 12.0 |	
|4	      | -	    | -	    |-   |	- | 1.2 |	2.0	   | 12.0 |	
|5 	      | -	| -	|-	    | 0.8 | -	| 2.0 | 12.0	  | 
|6	      | -	| -	|-	  | - | -	|-	   | 1.0	| 
|7	      | -	| -	|-	  | - | -	|7.0	   | -	| 

We also assume that at most 200 tons of the product can be shipped between any two nodes. This is the common arc capacity. RedBrand wants to determine a minimum-cost shipping schedule.


We would like to find a most efficient shipment plan to minimize the cost of transporting tomatos to meet customer demands. This example shows how to use sparse representation to model the fact that not all nodes are connected.

## Model Formulation

### Indices

$i,j,k \in \{1..7\}$: Index of seven different bus nodes.

### Parameters

$A$: Set of tuples ($i,j$) where shipmentment can be made from node $i$ to node $j$.

$s_{i}$: Supply capacity of node $i$, $i\in \{1,2,3\}$.

$d_{j}$: Demand of customer $j$, $j\in \{6,7\}$.

$c_{ij}$: Unit shipping cost from node $i$ to node $j$, $(i,j) \in A$.

$T$: Common shippment capacity on any route. [$T$=200 tons].

### Decision Variables

$x_{ij}$: Tons of tomota to ship from node $i$ to node $j$, $(i,j) \in A$.


### Objective Function

- **Cost**. We want to minimize the total operating cost.


\begin{equation}
\text{Min}_{x_{ij}} \quad \sum_{(i,j) \in A} c_{ij}*x_{ij}
\tag{0}
\end{equation}

### Constraints

\begin{equation}
\sum_{i|(i,j) \in A} x_{ij} - \sum_{k|(j,k) \in A} x_{jk} \geq d_j \quad \forall j \in \{6,7\} \quad (\text{customer demand must be covered})
\tag{1}
\end{equation}

\begin{equation}
\sum_{j|(i,j) \in A} x_{ij} - \sum_{k|(k,i) \in A} x_{ki} \leq s_i \quad \forall i \in \{1,2,3\} \quad (\text{cannot exceed plant capacity})
\tag{2}
\end{equation}

\begin{equation}
\sum_{i|(i,j) \in A} x_{ij} - \sum_{k|(j,k) \in A} x_{jk} =0 \quad \forall j \in \{4,5\} \quad (\text{flow balancing for transit nodes})
\tag{3}
\end{equation}

\begin{equation}
x_{ij} \leq k \quad \forall (i,j) \in A \quad (\text{common shipment capacity})
\tag{4}
\end{equation}

\begin{equation}
x_{ij} \geq 0 \quad \forall (i,j) \in A \quad (\text{non-negative shipping})
\tag{5}
\end{equation}

---

## Python Implementation

We now import the Gurobi Python Module and other Python libraries.

In [ ]:
%pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 4.0 MB/s 


In [ ]:
from itertools import product
from math import sqrt, factorial
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.1.0 and Python 3.7.0

Set up the model and solve

In [ ]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('tomato shipping')

# indices for companies and routes
plant = [*range(0,3)]
customer = [*range(5,7)]
transit = [*range(3,5)]
node = [*range(0,7)]

# print(plant)
# print(customer)
# print(transit)


# shipping cost 
c = [[0.0,	5.0,	3.0,	5.0,	5.0,	20.0,	20.0],
    [9.0,	0.0,	9.0,	1.0,	1.0,	8.0,	15.0],
    [0.4,	8.0,	0.0,	1.0,	0.5,	10.0,	12.0],
    [0.0,	0.0,	0.0,	0.0,	1.2,	2.0,	12.0],
    [0.0,	0.0,	0.0,	0.8,	0.0,	2.0,	12.0],
    [0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	1.0],
    [0.0,	0.0,	0.0,	0.0,	0.0,	7.0,	0.0]]

d = [0.0, 0.0, 0.0, 0.0, 0.0, 400.0, 180.0]

s = [200.0, 300.0, 100.0, 0.0, 0.0, 0.0, 0.0]

T = 200

# Valid set of tuples
A = []
for i in node:
    for j in node:
        if c[i][j] > 0:
            tp = i,j
            A.append(tp)

# take a look at the set
# print(np.matrix(A))

# valid set of inbound routes for node j
AI = [] 
k = 0
for l in node:
    A_temp = []
    for i in node:
        for j in node:
            if c[i][j] > 0:
                if j==k:
                    tp = i,j
                    A_temp.append(tp)
    AI.append(A_temp) 
    k+=1               

# take a look at a sample
# print(np.matrix(AI[0]))

# valid set of outbound routes for node j
AO = [] 
k = 0
for l in node:
    A_temp = []
    for i in node:
        for j in node:
            if c[i][j] > 0:
                if i==k:
                    tp = i,j
                    A_temp.append(tp)
    AO.append(A_temp) 
    k+=1               

# take a look at a sample
# print(np.matrix(AO[0]))

# Build decision variables: where to assign company i to route j
x = m.addVars(A, vtype=GRB.CONTINUOUS, name='Ship')
    
# Objective function: Minimize total payroll cost
m.setObjective(gp.quicksum(c[i][j]*x[(i,j)] for i,j in A), GRB.MINIMIZE)
    
# Satisfy customer demand
demandConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AI[j]) - gp.quicksum(x[(j,k)] for j,k in AO[j]) >= d[j] for j in customer), 
                                      name='demandConstrs')

# Cannot exceed plant capacity
supplyConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AO[i]) - gp.quicksum(x[(k,i)] for k,i in AI[i]) <= s[i] for i in plant), 
                                      name='supplyConstrs')

# Blanacing for transit nodes
transitConstrs = m.addConstrs((gp.quicksum(x[(i,j)] for i,j in AI[j]) - gp.quicksum(x[(j,k)] for j,k in AO[j]) == 0 for j in transit), 
                                      name='transitConstrs')

# Shipment capacity constraint
capacityConstrs = m.addConstrs((x[(i,j)] <= T for i,j in A), 
                                      name='capacityConstrs')

# Run optimization engine
m.optimize()

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 33 rows, 26 columns and 78 nonzeros
Model fingerprint: 0x27e79db7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 4e+02]
Presolve removed 26 rows and 0 columns
Presolve time: 0.01s
Presolved: 7 rows, 26 columns, 52 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.250000e+01   0.000000e+00      0s
       7    3.2600000e+03   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.03 seconds (0.00 work units)
Optimal objective  3.260000000e+03


Take a look at the results of shipments

In [ ]:
#####################################################
#         Shipment results
#####################################################
    
print(f"\n\n___Optimal shipment from plants to customers________")
t_cost = 0
for i,j in A:
    if x[(i,j)].x > 0:
        if i<=2:
          s_node_type = "plant"
        elif i<=4:
          s_node_type = "warehouse"
        else:
          s_node_type = "customer"
        if j<=2:
          d_node_type = "plant"
        elif j<=4:
          d_node_type = "warehouse"
        else:
          d_node_type = "customer"
             
        print("Shipping %4d tons from %s %2d to %s %2d" % (x[(i,j)].x, s_node_type, i+1, d_node_type, j+1))
        t_cost += x[(i,j)].x*c[i][j]

print("The total cost of satisfying customer demand is $%5d" % (t_cost))
            



___Optimal shipment from plants to customers________
Shipping  180 tons from plant  1 to plant  3
Shipping  120 tons from plant  2 to warehouse  4
Shipping  180 tons from plant  2 to customer  6
Shipping   80 tons from plant  3 to warehouse  4
Shipping  200 tons from plant  3 to warehouse  5
Shipping  200 tons from warehouse  4 to customer  6
Shipping  200 tons from warehouse  5 to customer  6
Shipping  180 tons from customer  6 to customer  7
The total cost of satisfying customer demand is $ 3260


As we can see, all customer demands are satisfied with the utilization of warehouses. The plant capacity are not exceeded. Furthermore, the optimal solution uses transhipment strategy, for example, shipping from customer 6 to customer 7. 

---
##  Conclusion

In this example, we addressed the tomato shipping problem. We determined the optimal shipment  of tomatos from plants to customers: 
* Satisfy demand for each customer, 
* Minimize the total shipping cost,  
* Ensure plant capacities are not exceeded, and
* Utilize transhipment to reduce shipping cost.

A special technique in the model formulation is sparse reprentation, where we significantly reduce the number of decision variables by restricting the set of decisions to be on the valid routes only. This benefit becomes more significant as problem size grows.

This tomato shipment model can be used in many different settings to help companies make informed decisions about satisfying customer demands from a set of plants where there are transit stations allowing for transhipments.


##  References
[1] Sixty examples of business optimization models. https://ytyimin.github.io/tart-cherry/.

[2] Gurobi python reference. https://www.gurobi.com/documentation/

[3] This notebook is developed by Yimin Wang. If you have any comments or suggestions please contact yimin_wang@asu.edu